In [17]:
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from timeit import timeit
from pathlib import Path
import pickle
import pandas as pd
import numpy as np
from google.cloud import storage
import gzip
import json
from contextlib import closing
import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

from inverted_index_colab import *

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# These will already be installed in the testing environment so disregard the 
# amount of time (~1 minute) it takes to install. 
!pip install -q pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
!pip install -q graphframes
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
graphframes_jar = 'https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar'
spark_jars = '/usr/local/lib/python3.7/dist-packages/pyspark/jars'
!wget -N -P $spark_jars $graphframes_jar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.4 MB/s eta 0:00:00
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 21 not upgraded.
Need to get 36.6 MB of archives.
After this operation, 143 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading database ... 124016 files and directories currently installed.)
Pre

In [4]:
import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.ml.feature import Tokenizer, RegexTokenizer
from graphframes import *

In [5]:
# Initializing spark context
# create a spark context and session
conf = SparkConf().set("spark.ui.port", "4050")
conf.set("spark.jars.packages", "graphframes:graphframes:0.8.2-spark3.2-s_2.12")
sc = pyspark.SparkContext(conf=conf)
sc.addPyFile(str(Path(spark_jars) / Path(graphframes_jar).name))
spark = SparkSession.builder.getOrCreate()

In [6]:
spark

In [7]:
# Authenticate your user
# The authentication should be done with the email connected to your GCP account
from google.colab import auth
import signal

AUTH_TIMEOUT = 30

def handler(signum, frame):
  raise Exception("Authentication timeout!")

try:
  signal.signal(signal.SIGALRM, handler)
  signal.alarm(AUTH_TIMEOUT)
  auth.authenticate_user()
  signal.alarm(0)
except: 
  pass

Copying Data from GCP bucket

In [8]:
# Copy one wikidumps files 
import os
from pathlib import Path
from google.colab import auth
## RENAME the project_id to yours project id from the project you created in GCP 
project_id = 'assignment3-370416'
!gcloud config set project {project_id}

data_bucket_name = 'wikidata20210801_preprocessed'
try:
    if os.environ["wikidata_preprocessed"] is not None:
        pass  
except:
      !mkdir wikidumps
      !gsutil -u {project_id} cp gs://{data_bucket_name}/multistream1_preprocessed.parquet "wikidumps/" 

Updated property [core/project].
Copying gs://wikidata20210801_preprocessed/multistream1_preprocessed.parquet...
/ [1 files][316.7 MiB/316.7 MiB]                                                
Operation completed over 1 objects/316.7 MiB.                                    


In [9]:
# Making parquet file from wiki preprocessed data

from pathlib import Path 
import os

try:
    if os.environ["wikidata_preprocessed"] is not None:
      path = os.environ["wikidata_preprocessed"]+"/wikidumps/*"
except:
      path = "wikidumps/*"

parquetFile = spark.read.parquet(path)
parquetFile.show()

+---+--------------------+--------------------+--------------------+
| id|               title|                text|         anchor_text|
+---+--------------------+--------------------+--------------------+
| 12|           Anarchism|'''Anarchism''' i...|[{23040, politica...|
| 25|              Autism|'''Autism''' is a...|[{492271, Clinica...|
| 39|              Albedo|thumb|upright=1.3...|[{679294, diffuse...|
|290|                   A|'''A''', or '''a'...|[{290, See below}...|
|303|             Alabama|'''Alabama''' () ...|[{351590, Yellowh...|
|305|            Achilles|thumb|260px|Ancie...|[{1076007, potter...|
|307|     Abraham Lincoln|'''Abraham Lincol...|[{1827174, Alexan...|
|308|           Aristotle|'''Aristotle''' (...|[{1389981, bust},...|
|309|An American in Paris|'''''An American ...|[{13066, George G...|
|316|Academy Award for...|The '''Academy Aw...|[{39842, Academy ...|
|324|      Academy Awards|The '''Academy Aw...|[{649481, film in...|
|330|             Actrius|'''''Act

In [12]:
# Importing PageRank DF from GCP bucket

project_id = 'assignment3-370416'
!gcloud config set project {project_id}

data_bucket_name = 'ryan316597145'
try:
    if os.environ["pr"] is not None:
        pass  
except:
      !mkdir pr2
      !gsutil -u {project_id} cp gs://{data_bucket_name}/pr/part-00000-4e51edea-48bb-4366-97ed-4102b6ea0c7f-c000.csv.gz "pr2/" 

Updated property [core/project].
Copying gs://ryan316597145/pr/part-00000-4e51edea-48bb-4366-97ed-4102b6ea0c7f-c000.csv.gz...
- [1 files][ 62.9 MiB/ 62.9 MiB]                                                
Operation completed over 1 objects/62.9 MiB.                                     


In [14]:
# Uploading pageviews data from GCP bucket

project_id = 'assignment3-370416'
!gcloud config set project {project_id}

data_bucket_name = 'ryan316597145'
try:
    if os.environ["pageviews"] is not None:
        pass  
except:
      !mkdir pageviews
      !gsutil -u {project_id} cp gs://{data_bucket_name}/pageviews/pageviews-202108-user-4dedup.txt "pageviews/"
      !gsutil -u {project_id} cp gs://{data_bucket_name}/pageviews/pageviews-202108-user.pkl "pageviews/"

Updated property [core/project].
Copying gs://ryan316597145/pageviews/pageviews-202108-user-4dedup.txt...
| [1 files][286.4 MiB/286.4 MiB]                                                
Operation completed over 1 objects/286.4 MiB.                                    
Copying gs://ryan316597145/pageviews/pageviews-202108-user.pkl...
\
Operation completed over 1 objects/73.5 MiB.                                     


In [13]:
# Uploading association data from GCP bucket

project_id = 'assignment3-370416'
!gcloud config set project {project_id}

data_bucket_name = 'ryan316597145'
try:
    if os.environ["association"] is not None:
        pass  
except:
      !mkdir association
      !gsutil -u {project_id} cp gs://{data_bucket_name}/association/association_data.bin "association/"
      !gsutil -u {project_id} cp gs://{data_bucket_name}/association/association_data.bin.vectors.npy "association/" 

Updated property [core/project].
Copying gs://ryan316597145/association/association_data.bin...
- [1 files][ 65.1 MiB/ 65.1 MiB]                                                
Operation completed over 1 objects/65.1 MiB.                                     
Copying gs://ryan316597145/association/association_data.bin.vectors.npy...
/ [1 files][455.3 MiB/455.3 MiB]                                                
Operation completed over 1 objects/455.3 MiB.                                    


Building Inverted Indexes

In [28]:
# regex, stopwords and tokenizing helpers
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ["category", "references", "also", "external", "links", 
                    "may", "first", "see", "history", "people", "one", "two", 
                    "part", "thumb", "including", "second", "following", 
                    "many", "however", "would", "became"]
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
all_stopwords = english_stopwords.union(corpus_stopwords)

1) Body Index

In [29]:
doc_text_pairs = parquetFile.limit(1000).select("text", "id").rdd

def doc_length(text, id):
  ''' function that calculates the document length without stopwords. receives RDD
  par: text, id
  returns: list: [(doc id: doc length)] '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered = [i for i in tokens if i not in all_stopwords]
  result = [(id, len(filtered))]
  return result

doc_len = doc_text_pairs.flatMap(lambda x: doc_length(x[0], x[1]))
doc_length_dict = doc_len.collectAsMap()

def termcount(text):
  ''' function that calculates the term counts without stopwords. receives RDD
  par: text
  returns: counter.items() '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered = [i for i in tokens if i not in all_stopwords]
  counts = Counter(filtered)
  return counts.items()

# counting the normalizing factor for each doc_id
docs_normal = doc_text_pairs.map(lambda x: (x[1], 1/np.sqrt(np.sum([i[1]**2 for i in termcount(x[0])]))))
docs_normal_dict = docs_normal.collectAsMap()

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered = [i for i in tokens if i not in all_stopwords]
  result = [(i[0], (id, i[1])) for i in Counter(filtered).items()]
  return result

word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings.filter(lambda x: len(x[1])>10)

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))

w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  postingsmap = postings.map(lambda x: (token2bucket_id(x[0]),(x[0], x[1])))
  joinlist = postingsmap.groupByKey().mapValues(list)
  answer = joinlist.map(lambda x: (InvertedIndex.write_a_posting_list(x)))
  return answer

posting_locs_list = partition_postings_and_write(postings_filtered).collect()

# merge the posting locations into a single dict
super_posting_locs1 = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs1[k].extend(v)

In [30]:
# Create inverted index instance
Body_Index = InvertedIndex()
# Add docs normalizing factor
Body_Index.docs_normal = docs_normal_dict
# Adding the posting locations dictionary to the inverted index
Body_Index.posting_locs = super_posting_locs1
# Add the token - df dictionary to the inverted index
Body_Index.df = w2df_dict
# Add the total length of each document
Body_Index.doc_length = doc_length_dict
# Add number of docs in corpus
Body_Index.number_of_docs = len(doc_length_dict.keys())
# write the global stats out
Body_Index.write_index('.', 'Body_Index')

2) Title Index

In [31]:
doc_text_pairs = parquetFile.limit(1000).select("title", "id").rdd

def doc_length(text, id):
  ''' function that calculates the document length without stopwords. receives RDD
  par: text, id
  returns: dict: {doc id: doc length} '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered = [i for i in tokens if i not in all_stopwords]
  result = [(id, len(filtered))]
  return result

doc_len = doc_text_pairs.flatMap(lambda x: doc_length(x[0], x[1]))
doc_length_dict = doc_len.collectAsMap()

def word_count(text, id):
  ''' Count the frequency of each word in `text` (tf) that is not included in 
  `all_stopwords` and return entries that will go into our posting lists. 
  Parameters:
  -----------
    text: str
      Text of one document
    id: int
      Document id
  Returns:
  --------
    List of tuples
      A list of (token, (doc_id, tf)) pairs 
      for example: [("Anarchism", (12, 5)), ...]
  '''
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  filtered = [i for i in tokens if i not in all_stopwords]
  result = [(i[0], (id, i[1])) for i in Counter(filtered).items()]
  return result

word_counts = doc_text_pairs.flatMap(lambda x: word_count(x[0], x[1]))

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[0])

postings = word_counts.groupByKey().mapValues(reduce_word_counts)
postings_filtered = postings

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))

w2df = calculate_df(postings_filtered)
w2df_dict = w2df.collectAsMap()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS+124

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  postingsmap = postings.map(lambda x: (token2bucket_id(x[0]),(x[0], x[1])))
  joinlist = postingsmap.groupByKey().mapValues(list)
  answer = joinlist.map(lambda x: (InvertedIndex.write_a_posting_list(x)))
  return answer

posting_locs_list = partition_postings_and_write(postings_filtered).collect()

# merge the posting locations into a single dict
super_posting_locs2 = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs2[k].extend(v)

In [35]:
# Create inverted index instance
Title_Index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
Title_Index.posting_locs = super_posting_locs2
# Add the token - df dictionary to the inverted index
Title_Index.df = w2df_dict
# Add the total length of each document
Title_Index.doc_length = doc_length_dict
# write the global stats out
Title_Index.write_index('.', 'Title_Index')

3) Anchor Index

In [36]:
doc_text_pairs = parquetFile.limit(1000).rdd.filter(lambda x: x[3] !=[] and type(x[0]) == int).map(lambda row: (row[3]))
anchor_text = doc_text_pairs.flatMap(lambda x: x)
anchor_id_text = anchor_text.map(lambda x: (x.text, x.id))

def word_count(text, id):
    tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
    filtered = [i for i in tokens if i not in all_stopwords]
    result = [(i[0], (id, i[1])) for i in Counter(filtered).items()]
    return result

counts = anchor_id_text.flatMap(lambda x: word_count(x[0], x[1]))

def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  return sorted(unsorted_pl, key=lambda x: x[1])

postings = counts.groupByKey().mapValues(reduce_word_counts)

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  return postings.map(lambda x: (x[0], len(x[1])))

w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()

NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS+248

def partition_postings_and_write(postings):
  ''' A function that partitions the posting lists into buckets, writes out 
  all posting lists in a bucket to disk, and returns the posting locations for 
  each bucket. Partitioning should be done through the use of `token2bucket` 
  above. Writing to disk should use the function  `write_a_posting_list`, a 
  static method implemented in inverted_index_colab.py under the InvertedIndex 
  class. 
  Parameters:
  -----------
    postings: RDD
      An RDD where each item is a (w, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each item is a posting locations dictionary for a bucket. The
      posting locations maintain a list for each word of file locations and 
      offsets its posting list was written to. See `write_a_posting_list` for 
      more details.
  '''
  postingsmap = postings.map(lambda x: (token2bucket_id(x[0]),(x[0], x[1])))
  joinlist = postingsmap.groupByKey().mapValues(list)
  answer = joinlist.map(lambda x: (InvertedIndex.write_a_posting_list(x)))
  return answer

posting_locs_list = partition_postings_and_write(postings).collect()

# merge the posting locations into a single dict
super_posting_locs3 = defaultdict(list)
for posting_loc in posting_locs_list:
  for k, v in posting_loc.items():
    super_posting_locs3[k].extend(v)

In [37]:
# Create inverted index instance
Anchor_Index = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
Anchor_Index.posting_locs = super_posting_locs3
# Add the token - df dictionary to the inverted index
Anchor_Index.df = w2df_dict
# write the global stats out
Anchor_Index.write_index('.', 'Anchor_Index')

4) Titles' names Index

In [40]:
# Making dict for Title: key = wiki_id, value = Title
newrdd = parquetFile.limit(1000).rdd
keypair_rdd = newrdd.map(lambda x : (x[0],x[1]))
parquetFile_dict = keypair_rdd.collectAsMap()

# Build the Index
TitleNames_Index = InvertedIndex()
# Add the docs names
TitleNames_Index.doc_name = parquetFile_dict
# write the global stats out
TitleNames_Index.write_index('.', 'TitleNames_Index')

In [43]:
# download nltk stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [44]:
# Install a particular version of `google-cloud-storage` because (oddly enough) 
# the  version on Colab and GCP is old. A dependency error below is okay.
!pip install -q google-cloud-storage==1.43.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.6/106.6 KB 3.1 MB/s eta 0:00:00


In [45]:
# authenticate below for Google Storage access as needed
from google.colab import auth
auth.authenticate_user()

In [46]:
# install ngrok to emulate public IP / address
!wget -N https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip -O ngrok-stable-linux-amd64.zip
!unzip -u ngrok-stable-linux-amd64.zip

for details.

--2023-01-16 08:40:04--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 18.205.222.128, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|18.205.222.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  41.0MB/s    in 0.3s    

2023-01-16 08:40:05 (41.0 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [47]:
# TODO: sign up for an ngrok account
# then put your ngrok token below, uncomment, and execute
!./ngrok authtoken 2JuNufsZYaRj39yvtQptTaXhOML_5Lct2KMH8bo5ZgaMEFkD8

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [48]:
# install a ngrok python package and a version of flask that works with it in 
# colab
!pip -q install flask-ngrok
!pip -q install flask==0.12.2
!pip -q install flask_restful

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 KB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 KB 5.2 MB/s eta 0:00:00


In [55]:
# Importing the Indexes:
Body_Index = InvertedIndex.read_index('.', 'Body_Index')
Title_Index = InvertedIndex.read_index('.', 'Title_Index')
Anchor_Index = InvertedIndex.read_index('.', 'Anchor_Index')
TitleNames_Index = InvertedIndex.read_index('.', 'TitleNames_Index')

# Run the app

In [60]:
# you need to upload your implementation of search_app.py
import search_frontend as se
#from inverted_index_colab import *

In [61]:
# uncomment the code below and execute to reload the module when you make 
# changes to search_frontend.py (after you upload again).
import importlib
importlib.reload(se)

<module 'search_frontend' from '/content/search_frontend.py'>

In [62]:
from flask_ngrok import run_with_ngrok
run_with_ngrok(se.app) 
se.app.run()

INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://4d18-34-70-47-162.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Jan/2023 09:09:54] "GET /search?query=alcohol HTTP/1.1" 200 -


MultiReader read - locs [('156_000.bin', 30)]
MultiReader read - locs [('280_000.bin', 450)]
MultiReader read - locs [('32_000.bin', 8706)]


INFO:werkzeug:127.0.0.1 - - [16/Jan/2023 09:09:54] "GET /favicon.ico HTTP/1.1" 404 -


# Testing your app

Once your app is running you can query it. You can simply do that by navigating to the URL that ngrok gave you above or through code in a different python session. For example, once the frontend app is running, you can navigate to:
http://YOUR_SERVER_DOMAIN/search?query=hello+world where YOUR_SERVER_DOMAIN is something like XXXX-XX-XX-XX-XX.ngrok.io, which is printed above in Colab or that is your external IP on GCP.

The code below shows how to issue a query from python. This is also how our testing code will issue queries to your search engine, so make sure to test your search engine this way after you deploy it to GCP and before submission. Command line instructions for deploying your search engine to GCP are available at `run_frontend_in_gcp.sh`. Note that we will not only issue training queries to your search engine, but also test queries, i.e. queries that you've never seen before.

In [ ]:
import json

with open('queries_train.json', 'rt') as f:
  queries = json.load(f)

In [ ]:
def average_precision(true_list, predicted_list, k=40):
    true_set = frozenset(true_list)
    predicted_list = predicted_list[:k]
    precisions = []
    for i,doc_id in enumerate(predicted_list):        
        if doc_id in true_set:
            prec = (len(precisions)+1) / (i+1)            
            precisions.append(prec)
    if len(precisions) == 0:
        return 0.0
    return round(sum(precisions)/len(precisions),3)

In [ ]:
import requests
from time import time
# url = 'http://35.232.59.3:8080'
# place the domain you got from ngrok or GCP IP below. 
url = 'http://a4c8-34-125-41-251.ngrok.io'

qs_res = []
for q, true_wids in queries.items():
  print("true_wids",true_wids)
  duration, ap = None, None
  t_start = time()
  try:
    res = requests.get(url + '/search', {'query': q}, timeout=35)
    duration = time() - t_start
    print("status=", res.status_code)
    if res.status_code == 200:
      pred_wids, _ = zip(*res.json())
      ap = average_precision(true_wids, pred_wids)
      print("pred_wids",pred_wids)
  except:
    pass
  
  qs_res.append((q, duration, ap))

In [ ]:
from sklearn.metrics import average_precision_score
average_precision = [i[2] if type(i[2])== float else 0 for i in qs_res]
average_time = [i[1] if type(i[1])== float else 0 for i in qs_res]
map_at_40 = sum(average_precision) / len(average_precision)
avg_time = sum(average_time) / len(average_time)
print("MAP@40:", map_at_40, avg_time)